<a href="https://colab.research.google.com/github/juliawol/WB_Knowledge_Base/blob/main/WB_Triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned model with terminology
model = SentenceTransformer('/content/fine_tuned_model_with_triplets')

# Load the original chunks
chunks_data_path = '/content/chunks.csv'
chunks_df = pd.read_csv(chunks_data_path)
original_chunks = chunks_df['Chunk'].tolist()

# Compute embeddings for chunks once
chunk_embeddings = model.encode(original_chunks, convert_to_tensor=True)

# Load a cross-encoder model for re-ranking
tokenizer = AutoTokenizer.from_pretrained('DiTy/cross-encoder-russian-msmarco')
cross_encoder_model = AutoModelForSequenceClassification.from_pretrained('DiTy/cross-encoder-russian-msmarco')

# Function to embed text using the fine-tuned model
def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=True)

# Function to retrieve top-k relevant chunks
def find_relevant_chunks(question_embedding, top_k=5):
    cosine_similarities = cosine_similarity(question_embedding.cpu().numpy(), chunk_embeddings.cpu().numpy()).flatten()
    num_candidates = top_k * 10
    top_indices = cosine_similarities.argsort()[-num_candidates:][::-1]
    return [original_chunks[i] for i in top_indices]

# Function to re-rank candidates using the cross-encoder
def re_rank(question, candidate_chunks):
    inputs = tokenizer([question] * len(candidate_chunks), candidate_chunks, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        scores = cross_encoder_model(**inputs).logits.squeeze()
    ranked_indices = scores.argsort(descending=True)
    return [candidate_chunks[i] for i in ranked_indices]

# Combined function to find top-k relevant chunks with optional re-ranking
def find_relevant_chunks_with_reranking(question, top_k=5):
    question_embedding = embed_texts([question])
    candidate_chunks = find_relevant_chunks(question_embedding, top_k=top_k)
    ranked_chunks = re_rank(question, candidate_chunks) if len(candidate_chunks) > 1 else candidate_chunks
    return ranked_chunks[:top_k]

# Display function for readable output
def display_relevant_chunks(chunks):
    print("Transformer-Based Relevant Chunks:")
    for idx, chunk in enumerate(chunks, start=1):
        print(f"\nChunk {idx}:\n{chunk}")

# Example usage
question = "Как завершить приемку товара без штрихкода?"
relevant_chunks = find_relevant_chunks_with_reranking(question, top_k=5)
display_relevant_chunks(relevant_chunks)


Transformer-Based Relevant Chunks:

Chunk 1:
На товаре нет штрихкода при приемке: Способ 1  1. Если на товаре есть баркод, откройте раздел «Движение вещей» 2. Пикните баркод вещи — в программе отобразится приходная коробка и штрихкод товара  Способ 2  1. Запишите данные коробки, в которой был товар 2. Когда завершите приёмку, создайте обращение 3. Укажите номер коробки и баркод вещи — КЦ пришлют правильный штрихкод  Способ 3  1. После того как приняли и разложили все товары, откройте раздел «Движение вещей» 2. Выберите нужную дату: в окне отобразится операция «Завершение приёмки коробки» 3. Скопируйте номер из столбца «ШК / Стикер» 4. Перейдите в раздел «История ШК» и вставьте номер в пустое поле 5. Сравните вещь с карточкой товара. Если совпадает, примите товар вручную и напишите ШК на пакете. Если нет, напишите руководителю или в поддержку

Chunk 2:
Чтобы подготовить возврат к отправке: - Надёжно упакуйте товар, чтобы вещь не повредилась и не потерялась во время транспортировки. - Ес

In [ ]:
question = "Потерял коробку, что делать?"
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Ни в коем случае нельзя: - Возвращать товары в незапечатанных коробках - Использовать поврежденные коробки - Оставлять старые этикетки для возврата или подписывать номер коробки вручную. Из-за этого коробка может потеряться во время транспортировки. - Прикреплять товар скотчем поверх коробки. Если вещь не поместилась внутрь, нужно перенести её в другую коробку, где есть свободное место.

Chunk 2:
Если вы потеряете товар или коробку в пункте выдачи, отправите вещь на склад без штрихкода или товар не вернётся в сортировочный центр после возврата, программа посчитает это за недостачу. Из зарплаты удержат сумму в размере стоимости товара.

Chunk 3:
Если при приемке адрес на коробке не совпал,  не принимайте коробку на баланс офиса. Отложите её, запишите номер и адрес назначения, а затем отправьте информацию руководителю. Коробку, которую доставили по ошибке, передайте курьеру. Обязательно делайте это под камерами видеонаблюдения.  Если вы по оши

In [ ]:
question = "Кто решает, когда делать генеральную уборку?"
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Как часто проводить генеральную уборку, решает руководитель пункта. Обычно — от 1 раза в неделю до 1 раза в 1-2 месяца.  Что сделать:  - Выполнить все пункты из чек-листа «Ежедневная уборка» - Убрать паутину из углов - Отмыть ножи и ножницы от скотча, заменить лезвия - Протереть технику, антивандальный ящик - Протереть мебель - Протереть огнетушители и подставки под них - Постирать шторы из примерочных - Помыть коврики - Заказать расходники, если что-то закончилось

Chunk 2:
Носить бейдж на работе в ПВЗ — значит не просто соблюдать требование компании, но и проявлять заботу о покупателях.  Почему бейдж важен:  - Покупателям будет проще определить, кто менеджер ПВЗ, и обратиться к вам по имени — это персонализирует обслуживание и делает клиентский опыт приятнее. - Бейдж повышает безопасность в пункте выдачи: вам будет легче следить за тем, чтобы посторонние не проникли в рабочую зону.  

Chunk 3:
Провести инвентаризацию — значит проверить ост

In [ ]:
question = "Как настроить маршрутизатор ASUS?"
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Чтобы настроить регистратор на телефоне или планшете: 1. Подключите жёсткий диск по инструкции 2. Подключите провод питания и патч-корд от роутера в отдельный порт LAN, а камеры — в порты POE 3. Подключитесь к Wi-Fi. Зажмите пальцем название сети и выберите «Изменить сеть» или нажмите символ в виде карандаша в правом верхнем углу 4. Поставьте галочку в строке «Дополнительно» 5. Нажмите «Настройки IP» и выберите «Статический IP» или «Пользовательские» — откроются настройки статического IP 6. Введите данные в поля:  - IP-адрес: 192.168.1.2 - Шлюз: 192.168.1.1 - DNS 1: 8.8.8.8  7. Нажмите «Сохранить» 8. Во всплывающем окне «У этой сети Wi-Fi нет доступа в Интернет. Подключиться?» нажмите «Подключить» 9. Не обращайте внимание на статус подключения «Подключено (нет доступа в Интернет)» — так должно быть на этом этапе 10. Откройте браузер на планшете или телефоне и введите в адресную строку заводской IP адрес регистратора: 192.168.1.108 11. В поле

In [ ]:
question = "Можно ли считать QR-код на товаре как обычный штрихкод?"
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Если на товаре при приемке вместо ШК стикер с QR-кодом, сканируйте его так же, как и обычный штрихкод.

Chunk 2:
Если нет штрихкода, но нужно произвести возврат: 1. Введите номер телефона покупателя, чтобы открыть его в системе. 2. Найдите товар в списке по информации на бирке и внешнему виду вещи. 3. Проверьте, есть ли дефекты или следы носки. 4. Сравните бренд, размер и комплектацию с информацией в карточке товара. 5. На листке бумаги напишите вручную штрихкод — это нужно для того, чтобы на складе смогли идентифицировать товар и переупаковать его. 6. Приклейте листок со штрихкодом на пакет с товаром с помощью прозрачного скотча. 7. Введите штрихкод товара вручную и упакуйте его в коробку для возврата.

Chunk 3:
Если вы не можете определить ячейку, в которой лежит товар, введите штрихкод в строку поиска. Программа покажет номер ячейки и имя покупателя.

Chunk 4:
Как избежать вычета за порчу товара: Не ставьте отметки маркером или ручкой на 

In [ ]:
question = "Что будет, если я отправлю не тот товар на склад? "
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Если вы не заметите подмену и отправите на склад не тот товар, из зарплаты вычтут сумму в размере стоимости вещи.

Chunk 2:
Удержание «Неотправленный возврат в коробке на склад». Если вы отправляли товар на склад, но он потерялся в пути, проверьте историю штрихкода и выберите в заявке склад или СЦ, где товар сканировали в последней раз. - Удержание «Подмена товара». Просмотрите историю штрихкода: там будет статус «Неправильное вложение на переупаковке» и название склада или СЦ, где обнаружили подмену — туда нужно будет отправить заявку. - Удержание «Не принятый товар на полку». Если в приходной коробке не оказалось одного из товаров, нужно найти в истории штрихкода последний склад или СЦ, где его сканировали, и отправить туда заявку.

Chunk 3:
При хранении товара не нужно: - Маркировать товар. Если на коробке будут надписи ручкой или маркером, покупатель может пожаловаться и вернуть товар как бракованный. Тогда из выплаты менеджера удержат с

In [ ]:
question = "Что делать, если конфликт с покупателем?"
display_relevant_chunks(relevant_chunks)

Transformer-Based Relevant Chunks:

Chunk 1:
Покупатели не знают, как устроены процессы в Wildberries, поэтому направляют негатив на менеджера пункта, даже если ошибка произошла не по вашей вине.  Что делать,  если возник конфликт:  - Не поддавайтесь эмоциям, если покупатель проявляет агрессию и провоцирует. - Говорите спокойно и чётко, даже когда ситуация накаляется. - Не принимайте негатив на свой счёт. Постарайтесь как можно лучше разобраться в проблеме. Покажите искренний интерес, чтобы покупатель понимал, что ему хотят помочь.  Что делать:  - Узнайте имя. Обращайтесь к покупателю только так, как он представился. Если обсуждаете проблему при покупателе с сотрудником поддержки или руководством, не используйте местоимения «он» или «она» — называйте клиента по имени. - Выслушайте. Дайте покупателю высказаться и не перебивайте — так вы сможете понять контекст и вникнуть в суть проблемы. - Задайте вопросы. Когда покупатель закончит монолог, вежливо уточните то, что не поняли: «Я правиль

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the fine-tuned model
model = SentenceTransformer('/content/fine_tuned_model_with_triplets')

# Load the chunks to choose from and the ground truth data
chunks_data_path = '/content/chunks.csv'
eval_data_path = '/content/train_data1.csv'
chunks_df = pd.read_csv(chunks_data_path)
eval_data_df = pd.read_csv(eval_data_path)

# Ensure the ground truth columns are correctly named
eval_data_df = eval_data_df.rename(columns={'Chunk': 'Ground Truth Chunk'})

# Pre-compute embeddings for all chunks in chunks.csv
original_chunks = chunks_df['Chunk'].tolist()
chunk_embeddings = model.encode(original_chunks, convert_to_tensor=True)

# Function to calculate recall at k
def calculate_recall_at_k(model, eval_data, k=5):
    correct_at_k = 0
    total = 0

    for _, row in eval_data.iterrows():
        question = row['Question']
        ground_truth_chunk = row['Ground Truth Chunk']

        # Embed the question
        question_embedding = model.encode([question], convert_to_tensor=True)

        # Calculate cosine similarity between the question and all chunk embeddings
        cosine_similarities = cosine_similarity(question_embedding.cpu().numpy(), chunk_embeddings.cpu().numpy()).flatten()

        # Get top-k most similar chunks
        top_k_indices = cosine_similarities.argsort()[-k:][::-1]
        top_k_chunks = [original_chunks[i] for i in top_k_indices]

        # Check if the ground truth chunk is in the top-k retrieved chunks
        if ground_truth_chunk in top_k_chunks:
            correct_at_k += 1
        total += 1

    # Calculate recall at k
    recall_at_k = correct_at_k / total
    return recall_at_k

# Calculate recall@1, recall@3, and recall@5
recall_at_1 = calculate_recall_at_k(model, eval_data_df, k=1)
recall_at_2 = calculate_recall_at_k(model, eval_data_df, k=2)
recall_at_3 = calculate_recall_at_k(model, eval_data_df, k=3)
recall_at_5 = calculate_recall_at_k(model, eval_data_df, k=5)

print(f"Recall@1: {recall_at_1:.4f}")
print(f"Recall@2: {recall_at_2:.4f}")
print(f"Recall@3: {recall_at_3:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")


Recall@1: 0.4463
Recall@2: 0.5455
Recall@3: 0.6116
Recall@5: 0.6777
